# Naive RAG with Langchain

In this notebook, we implement the naive RAG (simplest approach) using langchain, we will use:

* Data: pdf files about the creation and development of SAAS businesses.

* Embedding model: We will use an open source embedding model from HuggingFace "BAAI/bge-small-en-v1.5" which gives relatively good performance (see HuggingFace embeddings model leaderboard [here](https://huggingface.co/spaces/mteb/leaderboard)).

* Vector store: local FAISS vector store.

* LLM: llama3-70b-8192 model with Groq API.

### Install dependencies

In [25]:
!pip -q install langchain langchain-groq faiss-cpu sentence_transformers pypdf

### Setup API keys

You must create a Groq api key from [here](https://wow.groq.com/)

In [ ]:
import os

os.environ['GROQ_API_KEY'] =  "gsk_xxxxxxxxx"

### Load data

All our PDF files are stored in the "/data" folder, we use the DirectoryLoader to load them.

Because the loaded documents are too big to fit in a single context, we'll split them into smaller text chunks with RecursiveCharacterTextSplitter.

A critical parameter that you must tune to get a good RAG performance is the chunk_size (set to 1024 below).

* While a large chunk size could give more context, it will introduce more noise and require more computational costs, and could also result in “Lost in the Middle” effect (when LLM context is too long it will forget what's in the middle and concentrate on the beginning and the end).

* On the other hand, a smaller chunk size will have less noise to the model but may not provide the full context necessary for the answer.

So we must pick the perfect size between both extremes.

The chunk_overlap paramter is used to keep some sort of continuity in the text chunks, and it can also be tuned.

In [29]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

loader = DirectoryLoader(
    "./data",
    loader_cls=PyPDFLoader,
    show_progress=True
)

documents = loader.load()
print(f"Loaded {len(documents)} docs")

100%|██████████| 2/2 [00:02<00:00,  1.44s/it]

Loaded 85 docs


In [33]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
text_chunks = text_splitter.split_documents(documents)

### Download Embedding model

In [34]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

### Convert to vectors and store to FAISS

In [35]:
from langchain_community.vectorstores import FAISS

# will automaticly convert given text chunks into vector using provided embeddings
# then we will store into FAISS DB
vectorstore = FAISS.from_documents(text_chunks, embeddings)

In [40]:
# this retriever will be used to pick chunks from vector store which have similarities with user query
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})

### Save DB locally

In [ ]:
vectorstore.save_local("faiss_index")

### Load local store

In [ ]:
new_vector_store = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

retriever = new_vector_store.as_retriever()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [41]:
# standard QA prompt
template = """You are a helpful assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that I don't know.
Question: {question}
Context: {context}
"""
prompt = ChatPromptTemplate.from_template(template)

In [42]:
# run llama3 using Groq
llm = ChatGroq(model="llama3-70b-8192", api_key=os.getenv("GROQ_API_KEY"))

# build retrieval chain using LCEL
# this will take the user query and generate the answer
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [44]:
result = rag_chain.invoke("How to get your first customers?")
print(result)

Based on the provided context, to get your first customers, here are four proven strategies:

1. **Create a landing page**: Start by buying a domain name and building a basic web page with a form for users to provide their contact information. Include social media links and your logo.

2. **Reach out to your immediate network**: Don't be afraid to start selling and seeking prospects.

Additionally, the context suggests that you should not wait until you have a finished product to get customers. You can pitch your product as a preorder and offer it for a steep discount to those willing to pre-purchase, including free setup and support.

Remember, the key is to **stop stalling, stop planning, and start selling**.


In [45]:
result = rag_chain.invoke("How to increase your sales efforts?")
print(result)

Based on the provided context, here are some tips to increase your sales efforts:

1. **Identify your authentic competitive advantage**: Leverage your strengths, whether it's writing, speaking, design, or anything else, to pitch your product.
2. **Start selling**: Stop planning and start taking action. Reach out to at least 10 people in the next 24 hours to pitch your product.
3. **Hustle**: Pick up the phone and boot up the computer to reach out to qualified prospects and pitch your service.
4. **Create a sales script**: Develop a script to help you stay focused and effective in your sales conversations.
5. **Establish a conversion funnel**: Set up a system to track and optimize your sales process.
6. **Optimize implementation and iterate**: Continuously improve your sales process by testing, tracking data, and talking to prospects.
7. **Increase online searchability**: Generate valuable content to increase your online visibility and attract potential customers.

Remember, sales is ab